# Bedrock - Prompting for Finance and Insurance domain

In this notebook, we include different example use cases for Finance and Insurance domain using Amazon Bedrock.

All the input prompts and the output from the models are logged into an output csv file

First we install a few libraries required in the notebook

In [4]:
#!unzip ../bedrock-preview-documentation/SDK/bedrock-python-sdk.zip -d /root/bedrock

#!pip install --upgrade pip
#!pip install scikit-learn seaborn

#!pwd
#!python3 -m pip install /root/bedrock/boto3-1.26.142-py3-none-any.whl
#!python3 -m pip install /root/bedrock/botocore-1.29.142-py3-none-any.whl

Create the output prompt-data folder

In [9]:
!mkdir -p prompt-data

Initializations

In [10]:
import boto3
import json
import csv
from datetime import datetime

bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-east-1',
 endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

output_csv_file = "./prompt-data/prompt-data-finance.csv"

List available Bedrock models

In [7]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '2f92f991-e03f-44be-9650-65e859f9bab3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 08 Jun 2023 14:13:56 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2f92f991-e03f-44be-9650-65e859f9bab3'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

Generic function to invoke Bedrock models and store the output

In [7]:
def call_bedrock(modelId, prompt_data, out_file):
    if 'amazon' in modelId:
        body = json.dumps({
            "inputText": prompt_data,
            "textGenerationConfig":
            {
                "maxTokenCount":4096,
                "stopSequences":[],
                "temperature":0,
                "topP":0.9
            }
        })
        #modelId = 'amazon.titan-tg1-large'
    elif 'anthropic' in modelId:
        body = json.dumps({
            "prompt": prompt_data,
            "max_tokens_to_sample": 4096,
            "stop_sequences":[],
            "temperature":0,
            "top_p":0.9
        })

        #modelId = 'anthropic.claude-instant-v1'
    elif 'ai21' in modelId:        
        body = json.dumps({
            "prompt": prompt_data,
            "maxTokens":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        })
        #modelId = 'ai21.j2-grande-instruct'
    elif 'stability' in modelId:
        body = json.dumps({"text_prompts":[{"text":prompt_data}]}) 
        #modelId = 'stability.stable-diffusion-xl'
    else:
        print('Parameter model must be one of titan, claude, j2, or sd')
        return
    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before)
    response_body = json.loads(response.get('body').read())

    if 'amazon' in modelId:
        response = response_body.get('results')[0].get('outputText')
    elif 'anthropic' in modelId:
        response = response_body.get('completion')
    elif 'ai21' in modelId:
        response = response_body.get('completions')[0].get('data').get('text')

    #Add interaction to the local CSV file...
    #column_name = ["timestamp", "modelId", "prompt", "response", "latency"] #The name of the columns
    data = [datetime.now(), modelId, prompt_data, response, latency] #the data
    with open(out_file, 'a') as f:
        writer = csv.writer(f)
        #writer.writerow(column_name)
        writer.writerow(data)
    
    return response, latency

## General Question Answering1

### Titan

In [125]:
# Works fine
prompt_data ="""
Generate a list of main drivers of a company's stock price considering the following data, and include a short description of each.

Country is US
List of recommendations:
"""

In [126]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

1. Economic indicators such as inflation, interest rates, and GDP growth rate. These indicators can provide insight into the overall health of the economy and its impact on corporate earnings.

2. Industry trends: Monitoring industry trends can provide important information about the demand for a company's products or services and its competitive position within the industry.

3. Corporate governance practices, such as board structure, executive compensation, and shareholder rights. These practices can impact a company's reputation and financial performance.

4. Financial performance indicators such as revenue, earnings, and cash flow. These indicators can provide insight into a company's financial health and its ability to generate profits.

5. Economic events such as natural disasters, political events, and trade conflicts can impact a company's financial performance and stock price.

6. Stock market sentiment and investor sentiment can impact a company's stock price. These factors c

### Claude

In [101]:
# Works fine
prompt_data ="""User:
Generate a list of key drivers of a company's stock price considering the following data, and include a short description of each.
Country is US
Assistant:
"""

In [102]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

 Here is a list of key drivers of a company's stock price in the US, along with a short description of each:

1. Earnings growth - As a company's earnings and profits grow over time, investors are willing to pay a higher price for the stock. Earnings growth signals a healthy, growing company. 

2. Interest rates - Lower interest rates make stocks more attractive to investors relative to fixed income investments like bonds. As rates rise, stocks become slightly less attractive. Changes in rates impact the broader stock market and individual companies.

3. Economic growth - A strong, growing economy bodes well for most companies and the stock market overall. When the economy is expanding, consumers and businesses are spending more, which drives revenue and earnings growth for companies. 

4. Industry trends - The performance and outlook of a company's industry or sector impacts the company's own growth prospects and stock price. Changes in technology, regulations, competition, and consum

### Jurassic-2

In [127]:
# Works fine
prompt_data ="""
Generate a list of main drivers of a company's stock price considering the following data, and include a short description of each.

Country is US
"maxTokens": 2500
"""

In [128]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

Here is a list of the top drivers of a company's stock price, based on the data you provided:

1. Revenue growth: A company's revenue growth is a measure of its ability to generate sales, and is a key indicator of its financial performance.
2. Profitability: A company's profitability is a measure of its ability to generate revenue and profit, and is a key indicator of its financial performance.
3. Earnings per share (EPS): A company's EPS is a measure of its profitability, and is a key indicator of its financial performance.
4. Dividend yield: A company's dividend yield is a measure of its ability to pay dividends to shareholders, and is a key indicator of its financial performance.
5. Market share: A company's market share is a measure of its share of the market, and is a key indicator of its financial performance.
6. Industry trends: The industry trends of a company's industry can affect its financial performance.
7. Macroeconomic conditions: Macroeconomic conditions, such as the sta

## General Question Answering 2

In [79]:
# Works fine
prompt_data ="""User:
Generate a list of main types of insurance considering the data in the <metadata></metadata> XML tags, and include a short description of each.
<metadata>
Country is US
List should include auto insurance
</metadata>
Assistant:
"""

In [80]:
# Works fine
prompt_data ="""User:
Generate a list of main types of insurance considering the following data, and include a short description of each.
Country is US
List should include auto insurance
Assistant:
"""

In [81]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

 Here is a list of major types of insurance in the US along with a short description of each:

• Auto insurance - Covers losses and liabilities arising from accidents and collisions involving vehicles. Required by law for all vehicle owners. 

• Health insurance - Covers medical and surgical expenses for the insured. Can be purchased privately or provided by employers.

• Homeowners insurance - Covers losses and damages to an individual's residence and personal property. Also provides liability coverage.

• Life insurance - Provides financial protection to beneficiaries upon the death of the insured. Can be term life or permanent life insurance.

• Business insurance - Covers losses and liabilities for businesses and commercial entities. Includes property, liability, business interruption, and other coverage.

• Disability insurance - Provides income replacement benefits for individuals unable to work due to injury or illness. Can be short-term or long-term coverage.  

• Renters insur

## General Question Answering 3

In [95]:
# Works fine
prompt_data ="""User:
Generate a list of main reasons why is insurance important considering the following data, and include a short description of the benefits of each.
Country is US
Assistant:
"""

In [96]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

 Here is a list of main reasons why insurance is important in the US, along with a short description of the benefits of each:

1. Protects against financial loss from unforeseen events: Insurance helps cover costs in the event of unexpected circumstances like accidents, injuries, illnesses, natural disasters, theft, etc. This protects people from potential financial hardship. 

2. Provides peace of mind: Insurance gives people reassurance that they will be covered financially in case of unforeseen events. This helps reduce anxiety and allows people to live with more security and stability.

3. Access to healthcare: Health insurance provides access to healthcare services like doctor visits, hospital stays, medical tests, surgeries, prescription drugs, etc. This allows people to get the care they need to maintain health and treat medical issues. 

4. Compliance with laws: Certain types of insurance like auto, home, and health insurance are required by law in the US. Having insurance help

## Question Answering with context

In [157]:
# Works fine
prompt_data ="""User:
Answer all questions one by one based on the following context:

XYZ, INC. reported consolidated net earnings of $8,299,000 in 2022 compared to $9,193,000 in 2021. This represents $3.92 earnings per share in 2022 compared to $4.24 earnings per share in 2021.
XYZ Board of Directors continued to repurchase shares through March 16, 2022 under the previously authorized stock repurchase initiative. 45,592 shares were repurchased in 2022 at an average purchase price of $42.97. XYZ repurchased approximately 2.1% of outstanding shares, providing greater return on equity to owners of shares. XYZ ended the fiscal year with 2,104,717 shares outstanding.
On October 1, 2022, XYZ holding company of DEF Bank, successfully completed a transaction whereby XYZ acquired DEF Bank. DEF Bank provides a comprehensive suite of business, personal and retail banking products. At acquisition, DEF Bank had $239 million in assets, $203 million in net loans and $205 million in deposits. Combined assets at acquisition were $1.3 billion.
On December 15, 2022, XYZ rewarded shareholders with a cash dividend of $0.68 per share. XYZ has consistently provided shareholders an annual dividend since 1989.
Book value on December 31, 2022, was $44.41 compared to $49.51 on December 31, 2021. The Federal Reserve’s Federal Open Market Committee (FOMC) increased rates at the fastest pace since 1994, increasing rates seven times during the year; including four successive rate increases of 75 basis points. This rising interest rate environment led to unrealized loss positions in TR Bank’s bond portfolio, negatively impacting book value. Because TR Bank maintains a strong liquidity position and contingency funding plans, TR Bank has no plans to sell securities in an unrealized loss position.
XYZ’s shares closed the year at $41.51, a decrease of 2.67%. Bank stocks traded in line with the S&P 500 throughout 2022, with the S&P 500 bank index down approximately 24%. The S&P 500 was down 19%, its biggest annual percentage drop since 2008. 

Assistant:
What amount of assets had DEF Bank?
What was the closing price of XYZ's shares at the end of the year? 
"""

In [158]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)


Human: DEF Bank had $239 million in assets at the time of acquisition.
XYZ's shares closed the year at $41.51. 

 Inference time: 0:00:01.281398


## Summarisation

In [230]:
# Works fine
prompt_data ="""User:
Summarise the following content in three sentences.

XYZ, INC. reported consolidated net earnings of $8,299,000 in 2022 compared to $9,193,000 in 2021. This represents $3.92 earnings per share in 2022 compared to $4.24 earnings per share in 2021.
XYZ Board of Directors continued to repurchase shares through March 16, 2022 under the previously authorized stock repurchase initiative. 45,592 shares were repurchased in 2022 at an average purchase price of $42.97. XYZ repurchased approximately 2.1% of outstanding shares, providing greater return on equity to owners of shares. XYZ ended the fiscal year with 2,104,717 shares outstanding.
On October 1, 2022, XYZ holding company of DEF Bank, successfully completed a transaction whereby XYZ acquired DEF Bank. DEF Bank provides a comprehensive suite of business, personal and retail banking products. At acquisition, DEF Bank had $239 million in assets, $203 million in net loans and $205 million in deposits. Combined assets at acquisition were $1.3 billion.
On December 15, 2022, XYZ rewarded shareholders with a cash dividend of $0.68 per share. XYZ has consistently provided shareholders an annual dividend since 1989.
Book value on December 31, 2022, was $44.41 compared to $49.51 on December 31, 2021. The Federal Reserve’s Federal Open Market Committee (FOMC) increased rates at the fastest pace since 1994, increasing rates seven times during the year; including four successive rate increases of 75 basis points. This rising interest rate environment led to unrealized loss positions in TR Bank’s bond portfolio, negatively impacting book value. Because TR Bank maintains a strong liquidity position and contingency funding plans, TR Bank has no plans to sell securities in an unrealized loss position.
XYZ’s shares closed the year at $41.51, a decrease of 2.67%. Bank stocks traded in line with the S&P 500 throughout 2022, with the S&P 500 bank index down approximately 24%. The S&P 500 was down 19%, its biggest annual percentage drop since 2008. 

Assistant:
"""

In [231]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

 Here is a summary in three sentences:

XYZ, INC. reported lower earnings and earnings per share in 2022 compared to 2021. XYZ continued share repurchases and paid dividends to shareholders. XYZ acquired DEF Bank in 2022 but saw a decrease in book value and stock price due to rising interest rates. 

 Inference time: 0:00:02.192841


In [227]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

XYZ, INC. reported consolidated net earnings of $8,299,000 in 2022 compared to $9,193,000 in 2021. This represents $3.92 earnings per share in 2022 compared to $4.24 earnings per share in 2021. XYZ Board of Directors continued to repurchase shares through March 16, 2022 under the previously authorized stock repurchase initiative. 45,592 shares were repurchased in 2022 at an average purchase price of $42.97. XYZ repurchased approximately 2.1% of outstanding shares, providing greater return on equity to owners of shares. XYZ ended the fiscal year with 2,104,717 shares outstanding. 

 Inference time: 0:00:03.555022


-----

In [4]:
prompt_data="""
Context: Actua como un agente de servicio al cliente de un banco. Responde en catalán.
Human: Considera la conversación telefónica en los tags XML <conversacion></conversacion> y recomienda la próxima respuesta que debe dar el agente de servicio al cliente.

<conversacion>
Agent: Buenos días, gracias por llamar a CaixaBank, ¿cómo puedo ayudarle?
Customer: Hola, llamo porque mi tarjeta de crédito ha sido bloqueada
Agent: Primero necesito verificar unos datos de identidad, por favor indique su código
Customer: Mi código es ********
Agent: Gracias. Ahora por favor indíqueme por qué piensa que su tarjeta ha sido bloqueada
Customer: He intentado hacer un pago y no lo ha aceptado
</conversacion>

Assistant:
"""

In [11]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data, output_csv_file)
print(response, "\n\n", "Inference time:", latency)

 La próxima respuesta que recomendaría dar el agente de servicio al cliente es:

Disculpe las molestias. Comprobaré el estado de su tarjeta de crédito en nuestro sistema. Por favor, espere un momento mientras reviso su cuenta. 

 Inference time: 0:00:02.783930
